In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import matplotlib as mpl
from matplotlib import rc
rc('text',usetex=False)
rc('font',**{'family':'serif','serif':['Times New Roman'],'size':8})
rc('mathtext',fontset='cm')
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'svg'
% config InlineBackend.rc = {'figure.figsize': (5,3.5)}

In [3]:
import numpy as np
from numpy import sqrt,log,array as ar
import numpy.linalg as la
import cvxpy as cvx
from helper import *
from cd.model.utility import LipschitzExpUtility as leu, RiskNeutralUtility as rnu
from cd.model.distrs import RademacherDistribution,UniformDistribution
import cd.model.synth_data as synth
import scaler
from itertools import product,zip_longest
from scipy.optimize import curve_fit

## Initialisation des constantes

In [5]:
u = leu(1)
bar_p = 50
ps = np.arange(1,bar_p+1)
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(bar_p)]

In [6]:
corvecs = np.zeros((3,bar_p))
corvecs[0] = 0.95/np.sqrt(bar_p) * np.ones(bar_p)
corvecs[1,0] = 0.95

In [7]:
Ms = [synth.GaussianMarket(Xs,R,corr_vector=corvec) for corvec in corvecs]
t_trues = [M.sample_t(5_000) for M in Ms]

In [39]:
xticks = [1]+list(range(5,55,5))

## Progression de EU*

In [8]:
Ms = [synth.GaussianMarket(Xs,R,corr_vector=corvec) for corvec in corvecs]
t_trues = [M.sample_t(5_000) for M in Ms]

In [9]:
q_trues = [[solve(t_true[:,:p],u=u,lamb=0)[0] for p in ps] for t_true in t_trues]
q_trues = np.array([np.array(list(zip_longest(*q_true,fillvalue=0))).T for q_true in q_trues])

In [10]:
eu_trues = np.array([[u(t_true@q_true).mean() for q_true in q_true0] for q_true0,t_true in zip(q_trues,t_trues)])

In [14]:
fig,ax = plt.subplots(1)
ax.plot(ps,eu_trues[1,:],'C0',label="Information concentrée")
ax.plot(ps,eu_trues[0,:],'C0-s',ms=4,markevery=4,label="Information dispersée")
ax.plot(ps,eu_trues[2,:],'C0-o',ms=4,markevery=4,label='Aucune information')
ax.axis(ymax=0.9)
ax.set_xticks([1]+list(range(5,55,5)));
ax.legend()
ax.set_xlabel('$p$')
ax.set_ylabel('Utilité')
plt.savefig('fig/pconst_euinforelative.pdf',bbox_inches='tight',pad_inches=0)


## Progression de la sous optimalité

In [31]:
m = 150
n = 10
tss = [create_sample(t_true,n=n,m=m) for t_true in t_trues]

In [0]:
q_trues[0,2]

In [52]:
k = 3
error = np.zeros((len(ps),k))
errorce = np.zeros((len(ps),k))
compout = np.zeros((len(ps),k))
bound = np.zeros((len(ps),k))
boundce = np.zeros((len(ps),k))
for i,p in enumerate(ps):
    if i%5 == 0: print(i)
    for j,(ts,t_true,eu_true) in enumerate(zip(tss,t_trues,eu_trues)):
        t = ts[:,:,:p]
        qs = solve(t,u,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        error[i,j] = np.percentile(eu_true[i] - outerror,95)
        errorce[i,j] = np.percentile(u.inverse(eu_true[i]) - u.inverse(outerror),95)
        bound[i,j] = bound_sou(n,sqrt(p),q_trues[j,i],lamb=1)
        boundce[i,j] = bound[i,j]*u.subinverse(u.inverse(eu_true[i]))
        idx = np.argmin(np.abs((eu_true[i] - outerror) - error[i,j]))
        compout[i,j] = outerror[idx]


45


40


35


30


25


20


15


10


5


0


In [116]:
fig,ax = plt.subplots(2,figsize=(6,8))
fig.subplots_adjust(hspace=.3)
pltu,pltce = ax

pltu.plot(ps[0],error[0,0],'k--',label='Bornes $(\delta = 5\%)$')
pltu.plot(ps,error[:,1],'C0-s',ms=4,markevery=4,label='Information concentrée')
pltu.plot(ps,error[:,0],'C0',ms=4,markevery=4,label='Information dispersée')
pltu.plot(ps,error[:,2],'C0-o',ms=4,markevery=4,label='Aucune information')
pltu.axis(ymax=1.1)
pltu.set_xticks(xticks)
pltu.set_title('a) Utilité')
pltu.set_ylabel('Erreur de sous optimalité (util)')
pltu.set_xlabel('$p$')
pltu.set_yticks(np.arange(0,0.61,0.1))
pltu.tick_params('y',colors='C0')
pltu.legend(loc='upper left')

pltu = pltu.twinx()
pltu.plot(ps,bound[:,0],'k--')
pltu.plot(ps[:],bound[:,1],'k--s',ms=4,markevery=4)
pltu.plot(ps[2:],bound[2:,2],'ko',ms=4,markevery=4)
pltu.axis(ymin=-3800)
pltu.set_yticks(np.arange(0,3001,500))

pltce.plot(ps[0],error[0,1],'k--s',ms=4,label='Borne $(\delta = 5\%)$')
pltce.plot(ps[0],error[0,1],'k--',label='Borne $(\delta = 5\%)$')
pltce.plot(ps[0],error[0,1],'k--o',ms=4,label='Borne $(\delta = 5\%)$')
pltce.plot(ps,error[:,1],'C0-s',ms=4,markevery=4,label='Information concentrée')
pltce.plot(ps,error[:,0],'C0',label='Information dispersée')
pltce.plot(ps,error[:,2],'C0-o',ms=4,markevery=4,label='Aucune information')
pltce.axis(ymax=1.1)
pltce.set_xticks(xticks)
pltce.set_title('b) Équivalent certain')
pltce.set_ylabel('$p$')
pltce.set_ylabel('Erreur de sous optimalité (Rendement)')
pltce.set_yticks(np.arange(0,0.61,0.1))
pltce.tick_params('y',colors='C0')
pltce.set_xlabel('$p$')
pltce.legend(loc='upper left',ncol=2)


pltce = pltce.twinx()
pltce.plot(ps,boundce[:,0],'k--')
pltce.plot(ps,boundce[:,1],'k--s',ms=4,markevery=4)
pltce.plot(ps,boundce[:,2],'k--o',ms=4,markevery=4)
pltce.axis(ymin=-9000)
pltce.set_yticks(np.arange(0,7001,1000));

plt.savefig('fig/pconst_so.pdf',bbox_inches='tight',pad_inches=0)
#

In [115]:
pltu
